Special thanks to 🚀 [Abhishek Thakur](https://www.linkedin.com/in/abhi1thakur/) for his incredible AutoTrain "Dream Booth" code, which made the entire process seamless.

In [ ]:
!pip install -U autotrain-advanced -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!autotrain setup --update-torch

2023-07-28 05:57:45.838206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Ins

In [7]:
import os
if not os.path.exists("images"): ##  you need to put 7-10 your images
  os.mkdir("images")
if not os.path.exists("images_refined"): ## new generated image will save here
  os.mkdir("images_refined")
if not os.path.exists("model"): ## Lora model will save
  os.mkdir("model")

## Training

* --resolution 512 or 1024 you can use based on GPU memory
* --use-8bit-adam : It will take less memory

In [ ]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output model/ \
--image-path images/ \
--prompt "photo of sks Vijay" \
--resolution 512 \
--batch-size 1 \
--num-steps 500 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--use-8bit-adam

## Image Generation using prompt

*  While training we have used photos of **sks vijay**
*  So in Image generation you can use **sks vijay** to tell model

In [ ]:
## you can use torch_dtype=torch.bfloat16(less memory will be used) or torch_dtype=torch.float16

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
import random

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.bfloat16,
)
pipe.to("cuda")
pipe.load_lora_weights("model/", weight_name="pytorch_lora_weights.safetensors")

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.bfloat16,
)
refiner.to("cuda")


prompt = "a portrait of sks vijay, pixar, cartoon, 3d, headshots, fantasy, 4k, uhd"

for seed in range(10):
    seed = random.randint(1,100)
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image.images[0]
    image = refiner(prompt=prompt, generator=generator, image=image)
    image = image.images[0]
    image.save(f"images_refined/{seed}.png")

In [ ]:
# prompt = "a portrait of sks vijay,approaching perfection, dynamic, highly detailed, artstation, concept art, smooth, sharp focus, illustration, art by Artemisia Gentileschi, a Handsome the cloud elf King busks on the streets of new york, playing guitar, singing"
# prompt ="Ignite your imagination with sks vijay and concept that blurs the boundaries of reality–a futuristic augmented reality gaming experience that blends the virtual and physical worlds seamlessly. Step into a new era of interactive entertainment, where players are immersed in an unrivaled adventure. Harnessing the power of advanced high-tech cameras, such as the Sony α7R IV and the Nikon Z9, every aspect is captured with remarkable precision. Version 3 of this groundbreaking concept transcends all expectations, infusing the virtual realm with astounding realism. As you enter this realm, the uplighting further elevates the immersion, transporting players to uncharted dimensions. With an aspect ratio of 16:9 and the absence of distracting text, the stage is set for a truly captivating experience. Brace yourself for a journey that pushes the boundaries of what is possible, as you become a protagonist in a world where the real and virtual seamlessly intertwine"
# prompt ="Immerse in the captivating portrait of a stunning Indian sks vijay Himachal Pradesh’s enchanting forests. This cinematic photoshoot, meticulously crafted, showcases his beauty with precision. Shot on a 25mm lens, depth of field and tilt blur create a compelling narrative. With settings like 1/1000 shutter speed, F/22 aperture, and calibrated white balance, the breathtaking 32k resolution reveals remarkable details. Super-resolution preserves nuances, while Pro Photo RGB enriches colors. Lighting techniques, from half rear to backlighting, accentuate elegance. Volumetric, Conte-Jour, and global illumination add depth. Scattering, reflections, and artistic elements enhance realism. Technical advancements like anti-aliasing and ambient occlusion refine quality. Prepare for an extraordinary blend of photography and CGI, where intricacy meets grace. –v 5.2 –style raw –ar 16:9"
# prompt ="a portrait of sks vijay Capture the essence of a sks vijay who has endured a profound personal transformation. Show them emerging from darkness into a new, enlightened existence. Use contrasting shadows and light to symbolize their journey from struggle to self-discovery"
# prompt ="Capture the protagonist, sks vijay as Harry Potter, in a heroic and determined pose. Use meticulous attention to detail to showcase his rugged charm and unwavering courage. The lighting should enhance the intensity in his eyes as he prepares to face the ultimate challenge. Add cinematic elements like floating magical artifacts around him to emphasize his wizarding prowess and the weight of his destiny"
# prompt ="A candid portrait of a street sks vijay  with a blurry backdrop of the cityscape. Use a Nikon D850 DSLR 4k camera with a 200mm lens and dramatic lighting to capture the passion and creativity of the artist"
# prompt ="sks Vijay is wearing an astronaut helmet, in the style of cyberpunk dystopia, 32k uhd, womancore, bronze and amber, dreamlike settings, gorecore, ivan albright –ar 107:53 –v 5"
# prompt ="sks Vijay approaching perfection, dynamic, highly detailed, artstation, concept art, smooth, sharp focus, illustration, art by Artemisia Gentileschi, a handsome, the cloud elf King busks on the streets of inda,background completly blur, singing"
